In [ ]:
pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3MB 4.6MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=556e15139f19921711a93713f144a93007d0e280f19a662ce6b0747695450d1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ahue1gk/wheels/fe/a0/79/b73d4a0c535b421b88fc7b393936b371fabbfeaf979eca4050
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
# Define Bullish Harami Pattern Test
def bullish_harami(yest_o, yest_c, tod_o, tod_c):
  if yest_o > yest_c and tod_o < tod_c and yest_o > tod_c and yest_c < tod_o:
    return True
  else:
    return False

# Define Bearish Engulfing Candle Test
def bearish_engulfing(yest_o, yest_c, tod_o, tod_c):
  if yest_o > yest_c and tod_o < tod_c and tod_o < yest_c and tod_c > 0.5*(yest_o + yest_c):
    return True
  else:
    return False

In [ ]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
from pandas_datareader import data as web

In [ ]:
def check(stock, start_date, end_date):
  yf.pdr_override()
  df = web.get_data_yahoo(stock, data_source='yahoo', start=start_date, end=end_date)

  df['tod_o'] = df['Open']
  df['tod_c'] = df['Close']
  df['yest_o'] = df['Open'].shift(1)
  df['yest_c'] = df['Close'].shift(1)

  # Add ATR 20 Value to df
  df['max_diff'] = pd.Series()
  df['h_l'] = df['High'] - df['Low']
  df['h_c'] = (df['High'] - df['Close']).abs()
  df['l_c'] = (df['Low'] - df['Close']).abs()
  df['max_diff'] = df[['h_l', 'h_c', 'l_c']].max(axis = 1)
  df['ATR20'] = df['max_diff'].rolling(window = 20).mean()
  df['ATR20'] = df['ATR20'].shift(1)
  
  df = df.iloc[20:,[0, 1, 6, 7, 8, 9, 14]]

  df['buy'] = pd.Series()
  df['sell'] = pd.Series()

  last_pos = 0
  buy_pos = 0
  last_funds = 100000.0

  harami_flag = False
  bear_engulf_flag = False
  stop_loss = np.nan
  target_price = np.nan

  for index, row in df.iterrows():
    if (harami_flag or bear_engulf_flag) and last_pos == 0:
      row['buy'] = row['tod_o']
      row['sell'] = np.nan
      row['funds'] = last_funds
      last_pos = int(last_funds / row['tod_o'])
      target_price = row['tod_o'] + 3 * row['ATR20']
      stop_loss = row['tod_o'] - 3 * row['ATR20']
    elif last_pos > 0 and row['yest_c'] > target_price:
      row['buy'] = np.nan
      row['sell'] = row['tod_o']
      last_funds = last_pos * row['tod_o']
      row['funds'] = last_funds
      last_pos = 0
    elif last_pos > 0 and row['yest_c'] < stop_loss:
      row['buy'] = np.nan
      row['sell'] = row['tod_o']
      last_funds = last_pos * row['tod_o']
      row['funds'] = last_funds
      last_pos = 0
    elif last_pos > 0:
      row['buy'] = np.nan
      row['sell'] = np.nan
      last_funds = last_pos * row['tod_c']
      row['funds'] = last_funds
    else:
      row['buy'] = np.nan
      row['sell'] = np.nan
      row['funds'] = last_funds

    harami_flag = bullish_harami(row['yest_o'],	row['yest_c'], row['tod_o'],	row['tod_c'])
    bear_engulf_flag = bearish_engulfing(row['yest_o'],	row['yest_c'], row['tod_o'],	row['tod_c'])

  df = df.dropna(subset = ['buy', 'sell'], how = 'all')

  df['company'] = stock
  df['target'] = df['buy'] + 3 * df['ATR20']
  df['stoploss'] = df['buy'] - 3 * df['ATR20']
  print(df.columns)

  return df.iloc[:, -5:]

In [ ]:
start_date = (datetime.datetime.now() - datetime.timedelta(days = 50)).strftime("%Y-%m-%d")
end_date = (datetime.datetime.now()).strftime("%Y-%m-%d")

niftynext50 = ['ABBOTINDIA', 'ACC', 'ADANIENT', 'ADANIGREEN', 'ADANITRANS', 'ALKEM', 'AMBUJACEM', 'APOLLOHOSP', 'AUROPHARMA', 'DMART', 'BAJAJHLDNG', 'BANDHANBNK', 'BERGEPAINT', 'BIOCON', 'BOSCHLTD', 'CADILAHC', 'COLPAL', 'DABUR', 'DLF', 'GAIL', 'GODREJCP', 'HAVELLS', 'HDFCAMC', 'HINDPETRO', 'ICICIGI', 'ICICIPRULI', 'IGL', 'INDUSTOWER', 'NAUKRI', 'INDIGO', 'JUBLFOOD', 'LTI', 'LUPIN', 'MARICO', 'MOTHERSUMI', 'MRF', 'MUTHOOTFIN', 'NMDC', 'PETRONET', 'PIDILITIND', 'PEL', 'PGHH', 'PNB', 'SBICARD', 'SIEMENS', 'TORNTPHARM', 'UBL', 'MCDOWELL-N', 'VEDL', 'YESBANK']
companies = ['GRASIM.NS', 'TITAN.NS', 'KOTAKBANK.NS', 'TATACONSUM.NS', 'HDFCLIFE.NS', 'NTPC.NS', 'HINDALCO.NS', 'ITC.NS', 'BAJAJFINSV.NS', 'BAJAJ-AUTO.NS', 'ULTRACEMCO.NS', 'BAJFINANCE.NS',	'NESTLEIND.NS','BHARTIARTL.NS','BRITANNIA.NS', 'SHREECEM.NS',	'HEROMOTOCO.NS', 'MARUTI.NS', 'CIPLA.NS', 'TATASTEEL.NS', 'ONGC.NS', 'ICICIBANK.NS', 'LT.NS', 'COALINDIA.NS', 'RELIANCE.NS', 'WIPRO.NS', 'TCS.NS', 'TECHM.NS', 'IIFLWAM.NS', 'TATASTLBSL.NS', 'NH.NS', 'OIL.BO', 'DEEPAKNI.BO', 'AMBER.NS', 'PHOENIXLTD.BO', 'ROUTE.NS', 'PGHL.NS', 'GRANULES.BO', 'FORTIS.BO', 'RATNAMANI.NS', 'NBCC.NS','BIRLACORPN.NS', 'ERIS.NS', 'INTELLECT.NS', 'WABCOINDIA.BO', 'WELSPUNIND.NS', 'CGCL.NS', 'BBTC.NS', 'SOLARINDS.BO', 'CYIENT.NS', 'MCX.BO', 'IIFL.NS', 'KAJARIACER.BO']
for i in niftynext50:
  # For nifty next 50
  stock = i + '.NS'
  # For sp500 or nifty50
  #stock = i
  df = check(stock, start_date, end_date)
  df.to_csv('signals.csv', mode='a', header=False)

[*********************100%***********************]  1 of 1 completed
Index(['Open', 'High', 'tod_o', 'tod_c', 'yest_o', 'yest_c', 'ATR20', 'buy',
       'sell', 'company', 'target', 'stoploss'],
      dtype='object')
[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added 

Index(['Open', 'High', 'tod_o', 'tod_c', 'yest_o', 'yest_c', 'ATR20', 'buy',
       'sell', 'company', 'target', 'stoploss'],
      dtype='object')
[*********************100%***********************]  1 of 1 completed

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/pyt


Index(['Open', 'High', 'tod_o', 'tod_c', 'yest_o', 'yest_c', 'ATR20', 'buy',
       'sell', 'company', 'target', 'stoploss'],
      dtype='object')
[*********************100%***********************]  1 of 1 completed


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added 

[*********************100%***********************]  1 of 1 completed
Index(['Open', 'High', 'tod_o', 'tod_c', 'yest_o', 'yest_c', 'ATR20', 'buy',
       'sell', 'company', 'target', 'stoploss'],
      dtype='object')
[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added 

Index(['Open', 'High', 'tod_o', 'tod_c', 'yest_o', 'yest_c', 'ATR20', 'buy',
       'sell', 'company', 'target', 'stoploss'],
      dtype='object')
[*********************100%***********************]  1 of 1 completed
Index(['Open', 'High', 'tod_o', 'tod_c', 'yest_o', 'yest_c', 'ATR20', 'buy',
       'sell', 'company', 'target', 'stoploss'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [ ]:
signals = pd.read_csv('signals.csv', names = ['Date', 'Buy', 'xyz', 'Company', 'Target', 'StopLoss'])
signals.drop('xyz', axis = 1, inplace = True)
signals = signals.sort_values(by = 'Date')
signals

,Date,Buy,Company,Target,StopLoss
30,2021-06-24,82198.000000,MRF.NS,87182.736719,77213.263281
20,2021-06-24,1545.000000,ICICIGI.NS,1667.302460,1422.697540
42,2021-06-25,678.000000,MCDOWELL-N.NS,733.845026,622.154974
9,2021-06-25,813.000000,BERGEPAINT.NS,860.887518,765.112482
21,2021-06-28,620.000000,ICICIPRULI.NS,667.550009,572.449991
27,2021-06-28,1158.099976,LUPIN.NS,1237.014990,1079.184961
31,2021-06-28,1490.050049,MUTHOOTFIN.NS,1640.365045,1339.735052
32,2021-06-28,179.300003,NMDC.NS,199.947502,158.652504
18,2021-06-28,300.500000,HINDPETRO.NS,324.117502,276.882498
36,2021-06-28,2425.100098,PEL.NS,2726.997583,2123.202612
